## Mga Classifier ng Lutuin 2

Sa ikalawang aralin ng klasipikasyon, tatalakayin natin ang `mas maraming paraan` upang iklasipika ang categorical na data. Matutunan din natin ang mga epekto ng pagpili ng isang classifier kumpara sa iba.

### [**Pre-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Paunang Kaalaman**

Inaakala namin na natapos mo na ang mga naunang aralin dahil magpapatuloy tayo sa ilang mga konsepto na natutunan natin dati.

Para sa araling ito, kakailanganin natin ang mga sumusunod na package:

-   `tidyverse`: Ang [tidyverse](https://www.tidyverse.org/) ay isang [koleksyon ng mga R package](https://www.tidyverse.org/packages) na idinisenyo upang gawing mas mabilis, mas madali, at mas masaya ang data science!

-   `tidymodels`: Ang [tidymodels](https://www.tidymodels.org/) framework ay isang [koleksyon ng mga package](https://www.tidymodels.org/packages/) para sa pagmomodelo at machine learning.

-   `themis`: Ang [themis package](https://themis.tidymodels.org/) ay nagbibigay ng Extra Recipes Steps para sa pag-aayos ng hindi balanseng data.

Maaari mong i-install ang mga ito gamit ang:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Bilang alternatibo, ang script sa ibaba ay nagche-check kung mayroon ka ng mga kinakailangang package upang makumpleto ang module na ito at ini-install ang mga ito para sa iyo kung sakaling wala pa.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Isang Mapa ng Klasipikasyon**

Sa ating [nakaraang aralin](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), sinubukan nating sagutin ang tanong: paano tayo pipili sa pagitan ng iba't ibang modelo? Sa malaking bahagi, nakadepende ito sa mga katangian ng datos at sa uri ng problemang nais nating lutasin (halimbawa, klasipikasyon o regresyon?).

Noong nakaraan, natutunan natin ang iba't ibang opsyon na mayroon ka kapag nagka-klasipika ng datos gamit ang cheat sheet ng Microsoft. Ang Machine Learning framework ng Python, Scikit-learn, ay nag-aalok ng katulad ngunit mas detalyadong cheat sheet na makakatulong pa upang mas paliitin ang pagpipilian mo ng mga estimator (isa pang termino para sa mga classifier):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Tip: [bisitahin ang online na mapa](https://scikit-learn.org/stable/tutorial/machine_learning_map/) at mag-click sa mga landas upang mabasa ang dokumentasyon.
>
> Ang [Tidymodels reference site](https://www.tidymodels.org/find/parsnip/#models) ay nagbibigay din ng mahusay na dokumentasyon tungkol sa iba't ibang uri ng modelo.

### **Ang plano** 🗺️

Ang mapa na ito ay napaka-kapaki-pakinabang kapag malinaw na ang iyong pagkaunawa sa data, dahil maaari kang 'maglakad' sa mga landas nito patungo sa isang desisyon:

-   Mayroon tayong \>50 na sample

-   Gusto nating magpredikta ng isang kategorya

-   Mayroon tayong labeled na data

-   Mayroon tayong mas kaunti sa 100K na sample

-   ✨ Maaari tayong pumili ng Linear SVC

-   Kung hindi ito gumana, dahil numeric ang ating data

    -   Maaari nating subukan ang ✨ KNeighbors Classifier

        -   Kung hindi ito gumana, subukan ang ✨ SVC at ✨ Ensemble Classifiers

Napaka-kapaki-pakinabang ng landas na ito upang sundan. Ngayon, simulan na natin gamit ang [tidymodels](https://www.tidymodels.org/) modelling framework: isang pare-pareho at flexible na koleksyon ng mga R packages na ginawa upang hikayatin ang mahusay na estadistikal na praktis 😊.

## 2. Hatiin ang data at harapin ang hindi balanseng data set.

Mula sa ating mga nakaraang aralin, natutunan natin na mayroong hanay ng mga karaniwang sangkap sa iba't ibang lutuin. Gayundin, may hindi pantay na distribusyon sa bilang ng mga lutuin.

Aayusin natin ito sa pamamagitan ng:

-   Pag-drop ng mga pinakakaraniwang sangkap na nagdudulot ng kalituhan sa pagitan ng magkakaibang lutuin, gamit ang `dplyr::select()`.

-   Paggamit ng isang `recipe` na nagpoproseso ng data upang ihanda ito para sa pagmomodelo sa pamamagitan ng pag-aaplay ng isang `over-sampling` algorithm.

Tinalakay na natin ang mga ito sa nakaraang aralin kaya dapat madali na ito 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Paano Harapin ang Hindi Balanseng Datos

Ang hindi balanseng datos ay madalas na may negatibong epekto sa performance ng modelo. Maraming modelo ang mas mahusay kapag pantay ang bilang ng mga obserbasyon, kaya't nahihirapan ang mga ito kapag hindi balansado ang datos.

May dalawang pangunahing paraan upang harapin ang hindi balanseng datos:

-   magdagdag ng mga obserbasyon sa minority class: `Over-sampling` halimbawa, gamit ang SMOTE algorithm na synthetically gumagawa ng mga bagong halimbawa ng minority class gamit ang mga pinakamalapit na kapitbahay ng mga kasong ito.

-   magtanggal ng mga obserbasyon mula sa majority class: `Under-sampling`

Sa nakaraang aralin, ipinakita namin kung paano harapin ang hindi balanseng datos gamit ang isang `recipe`. Ang recipe ay maituturing na isang plano na naglalarawan kung anong mga hakbang ang dapat gawin sa isang datos upang maihanda ito para sa pagsusuri. Sa ating kaso, nais nating magkaroon ng pantay na distribusyon ng bilang ng ating mga lutuin para sa ating `training set`. Sige, simulan na natin.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Ngayon, handa na tayong mag-train ng mga modelo 👩‍💻👨‍💻!

## 3. Higit pa sa multinomial regression models

Sa nakaraang aralin, tinalakay natin ang multinomial regression models. Tuklasin natin ang mas flexible na mga modelo para sa classification.

### Support Vector Machines

Sa konteksto ng classification, ang `Support Vector Machines` ay isang machine learning technique na naglalayong makahanap ng isang *hyperplane* na "pinakamahusay" na naghihiwalay sa mga klase. Tingnan natin ang isang simpleng halimbawa:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ hindi naghihiwalay ng mga klase. H2~ naghihiwalay, ngunit may maliit na margin lamang. H3~ naghihiwalay ng mga klase gamit ang pinakamalaking margin.

#### Linear Support Vector Classifier

Ang Support-Vector clustering (SVC) ay bahagi ng pamilya ng Support-Vector machines na mga teknik sa ML. Sa SVC, ang hyperplane ay pinipili upang tama na maihiwalay ang `karamihan` sa mga obserbasyon sa training, ngunit `maaaring magkamali` sa ilang obserbasyon. Sa pamamagitan ng pagpayag na ang ilang puntos ay nasa maling panig, nagiging mas matatag ang SVM laban sa mga outlier kaya mas mahusay ang generalization sa bagong data. Ang parameter na nagre-regulate sa paglabag na ito ay tinatawag na `cost` na may default na halaga na 1 (tingnan ang `help("svm_poly")`).

Gumawa tayo ng linear SVC sa pamamagitan ng pag-set ng `degree = 1` sa isang polynomial SVM model.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Ngayon na naitala na natin ang mga hakbang sa preprocessing at ang detalye ng modelo sa isang *workflow*, maaari na tayong magpatuloy sa pagsasanay ng linear SVC at suriin ang mga resulta habang ginagawa ito. Para sa mga performance metrics, gumawa tayo ng isang metric set na magsusuri ng: `accuracy`, `sensitivity`, `Positive Predicted Value`, at `F Measure`.

> Ang `augment()` ay magdaragdag ng column(s) para sa mga prediksyon sa ibinigay na data.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Support Vector Machine

Ang support vector machine (SVM) ay isang pagpapalawig ng support vector classifier upang makapagbigay-daan sa isang hindi linyar na hangganan sa pagitan ng mga klase. Sa esensya, ginagamit ng SVM ang *kernel trick* upang palakihin ang feature space at makaangkop sa mga hindi linyar na relasyon sa pagitan ng mga klase. Isa sa mga tanyag at napaka-flexible na kernel function na ginagamit ng SVM ay ang *Radial basis function.* Tingnan natin kung paano ito magpapakita ng resulta sa ating datos.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Mas maganda 🤩!

> ✅ Pakitingnan:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> para sa karagdagang pagbabasa.

### Mga classifier ng Nearest Neighbor

Ang *K*-nearest neighbor (KNN) ay isang algorithm kung saan ang bawat obserbasyon ay hinuhulaan batay sa *pagkakapareho* nito sa ibang mga obserbasyon.

Subukan nating mag-fit ng isa sa ating data.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Mukhang hindi masyadong mahusay ang performance ng modelong ito. Marahil ang pagbabago sa mga argumento ng modelo (tingnan ang `help("nearest_neighbor")`) ay makakatulong upang mapabuti ang performance ng modelo. Siguraduhing subukan ito.

> ✅ Mangyaring tingnan:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> upang matuto nang higit pa tungkol sa mga *K*-Nearest Neighbors classifiers.

### Ensemble classifiers

Ang mga ensemble algorithm ay gumagana sa pamamagitan ng pagsasama-sama ng maraming base estimators upang makabuo ng isang optimal na modelo sa pamamagitan ng:

`bagging`: paglalapat ng isang *averaging function* sa isang koleksyon ng mga base models

`boosting`: pagbuo ng sunod-sunod na mga modelo na nagtatayo sa isa't isa upang mapabuti ang predictive performance.

Simulan natin sa pamamagitan ng pagsubok sa isang Random Forest model, na bumubuo ng malaking koleksyon ng mga decision tree at pagkatapos ay nag-aaplay ng isang averaging function para makabuo ng mas mahusay na kabuuang modelo.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Magaling na trabaho 👏!

Subukan din nating gumamit ng Boosted Tree model.

Ang Boosted Tree ay isang ensemble method na lumilikha ng serye ng sunud-sunod na decision trees kung saan ang bawat puno ay nakadepende sa resulta ng mga naunang puno sa layuning unti-unting bawasan ang error. Nakatuon ito sa mga timbang ng mga maling na-classify na item at ina-adjust ang fit para sa susunod na classifier upang maitama.

May iba't ibang paraan upang i-fit ang modelong ito (tingnan ang `help("boost_tree")`). Sa halimbawang ito, i-fi-fit natin ang Boosted trees gamit ang `xgboost` engine.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Pakitingnan:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Tinalakay ang modelong AdaBoost na isang magandang alternatibo sa xgboost.
>
> upang matuto pa tungkol sa Ensemble classifiers.

## 4. Extra - paghahambing ng maraming modelo

Nakagawa tayo ng medyo maraming modelo sa lab na ito 🙌. Maaari itong maging nakakapagod o mahirap kung kailangang gumawa ng maraming workflows mula sa iba't ibang set ng preprocessors at/o model specifications at pagkatapos ay isa-isang kalkulahin ang performance metrics.

Tingnan natin kung paano natin ito masosolusyunan sa pamamagitan ng paggawa ng isang function na magfi-fit ng listahan ng workflows sa training set at pagkatapos ay magbabalik ng performance metrics base sa test set. Gagamitin natin ang `map()` at `map_dfr()` mula sa [purrr](https://purrr.tidyverse.org/) package upang mag-apply ng mga function sa bawat elemento ng listahan.

> Ang mga [`map()`](https://purrr.tidyverse.org/reference/map.html) na function ay nagbibigay-daan sa iyo na palitan ang maraming for loops ng code na mas maikli at mas madaling basahin. Ang pinakamagandang lugar upang matutunan ang tungkol sa [`map()`](https://purrr.tidyverse.org/reference/map.html) na mga function ay ang [iteration chapter](http://r4ds.had.co.nz/iteration.html) sa R for data science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) na package ay nagbibigay-daan sa mga user na lumikha at madaling mag-fit ng maraming modelo, ngunit ito ay pangunahing idinisenyo upang gumana gamit ang mga resampling techniques tulad ng `cross-validation`, isang pamamaraan na hindi pa natin natatalakay.

## **🚀Hamunin**

Ang bawat isa sa mga teknik na ito ay may maraming mga parameter na maaari mong i-adjust, tulad ng `cost` sa SVMs, `neighbors` sa KNN, `mtry` (Randomly Selected Predictors) sa Random Forest.

Magsaliksik tungkol sa default parameters ng bawat isa at pag-isipan kung ano ang magiging epekto ng pag-aadjust ng mga parameter na ito sa kalidad ng modelo.

Upang malaman ang higit pa tungkol sa isang partikular na modelo at ang mga parameter nito, gamitin ang: `help("model")` halimbawa `help("rand_forest")`

> Sa praktika, karaniwan nating *ina-estima* ang *pinakamainam na mga halaga* para dito sa pamamagitan ng pagte-train ng maraming modelo sa isang `simulated data set` at sinusukat kung gaano kahusay ang performance ng mga modelong ito. Ang prosesong ito ay tinatawag na **tuning**.

### [**Post-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Review at Pag-aaral sa Sarili**

Maraming jargon sa mga araling ito, kaya maglaan ng oras upang balikan ang [listahang ito](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) ng mga kapaki-pakinabang na termino!

#### PASASALAMAT SA:

[`Allison Horst`](https://twitter.com/allison_horst/) para sa paggawa ng mga kamangha-manghang ilustrasyon na ginagawang mas welcoming at engaging ang R. Hanapin ang iba pang ilustrasyon sa kanyang [gallery](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) at [Jen Looper](https://www.twitter.com/jenlooper) para sa paggawa ng orihinal na bersyong Python ng module na ito ♥️

Masayang Pag-aaral,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Artwork ni @allison_horst</figcaption>



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
